In [1]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
    
#import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["AUTOGRAPH_VERBOSITY"] = "0"

In [2]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


datasetPathVideo =  configParser.get('COMMON', 'test_datasetPathVideo')
datasetPathFaces =  configParser.get('evaluate_imagen', 'test_datasetPathFaces')
datasetPathFeatures =  configParser.get('evaluate_imagen', 'test_datasetPathFeatures')
datasetPathGuides =  configParser.get('evaluate_imagen', 'test_datasetPathGuides')

datasetPathDatabase =  configParser.get('COMMON', 'test_datasetPathDatabase') + '/dataset.db'

ttwbdf =  int(configParser.get('evaluate_imagen', 'time_to_wait_before_deleting_files'))


cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))




In [3]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [4]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [5]:
audio_folder = output_folder = r'imagen_testing_folder' 
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)

In [6]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
begin_with_image_size = int(configParser.get('COMMON', 'begin_with_image_size'))
timesteps= int(configParser.get('COMMON', 'timesteps') )
audio_length_used =  configParser.get('evaluate_imagen', 'audio_length_used') 
model_filename =  configParser.get('evaluate_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('evaluate_imagen', 'openl3_mode')
folder =  configParser.get('evaluate_imagen', 'folder')
number_of_images =  configParser.get('evaluate_imagen', 'number_of_images')

In [7]:

boxBlurMin =  int(configParser.get('evaluate_imagen', 'boxBlurMin'))
boxBlurMax =  int(configParser.get('evaluate_imagen', 'boxBlurMax'))

gaussianBlurMin =  int(configParser.get('evaluate_imagen', 'gaussianBlurMin'))
gaussianBlurMax =  int(configParser.get('evaluate_imagen', 'gaussianBlurMax'))

pix_to_min =  int(configParser.get('evaluate_imagen', 'pix_to_min'))
pix_to_max =  int(configParser.get('evaluate_imagen', 'pix_to_max'))

fddfb =  int(configParser.get('evaluate_imagen', 'faceDetectionDeepFaceBackend'))
efvr =  float(configParser.get('evaluate_imagen', 'expandFaceVerticalRatio'))
efhr =  float(configParser.get('evaluate_imagen', 'expandFaceHorizontalRatio'))


In [8]:
import lancedb
uri = datasetPathDatabase
db = lancedb.connect(uri)

table = db.open_table("video")

In [9]:
videos = table.to_pandas()

In [10]:
videos_unique_users = videos.drop_duplicates('user', keep='first')
videos_unique_users


,id,video_path,face_path,user,blurred_face_path,features_path,vector,stage
0,0,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,None,id00002,None,None,[0.065377645],None
1,1,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,None,id00001,None,None,[0.25357106],None
2,2,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,None,id00006,None,None,[0.3160929],None
3,3,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,None,id00008,None,None,[0.68535125],None
4,4,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,None,id00010,None,None,[0.28327712],None
5,5,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,None,id00012,None,None,[0.6307946],None
6,6,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,None,id00009,None,None,[0.32150412],None
7,7,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,None,id00005,None,None,[0.17822036],None
8,8,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,None,id00004,None,None,[0.9230645],None
10,10,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,None,id00007,None,None,[0.70353085],None


In [11]:
try:
  db.drop_table("video_stage1")
except:
  print("")

In [12]:
import multiprocessing
multiprocessing.set_start_method('spawn')

In [13]:
from testing_imagen_speechbrain import extract_speechbrain_embeddings
from testing_imagen_pyannote_titanet import extract_pyannote_titanet_embeddings
from testing_imagen_audio_features import extract_audio_features
from testing_imagen_vision_transformer import extract_vision_transformer
from testing_imagen_face import extract_face
import pathlib
import shutil


2023-10-06 22:56:01.389164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[NeMo W 2023-10-06 22:56:05 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-10-06 22:56:05 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [15]:


i = 0
for index, row in videos_unique_users.iterrows():
    video_path = absPathVideo = row['video_path']


    # EXTRACT FACE, AGE, GENDER, ETHNICITY, FACE EMBEDDING
    absPathFace = absPathVideo.replace(datasetPathVideo,datasetPathFaces)
    absPathFace = os.path.splitext(absPathFace)[0]
    absPathFace = absPathFace + "_face_" + str(1) + ".png"
    #print(absPathFace)
    #Create Directory
    pathlib.Path(os.path.dirname(absPathFace)).mkdir(parents=True, exist_ok=True) 

    
    q = multiprocessing.Queue()

    try:
        proc = multiprocessing.Process(target=extract_face, args=(q,absPathVideo,
                                                                image_size,
                                                                fddfb,
                                                                output_folder,
                                                                efvr,efhr,))
        proc.start()
        proc.join()
    except:
        continue

    shutil.copy2(output_folder + "/" + "face.png", 
                 absPathFace) # complete target filename given
    
    image_guide_path = output_folder + "/" + "face.png"
    low_res_image_path = output_folder + "/" + "face.png"

    gender = q.get()
    if(gender == 'Error'):
        continue
    ethnicity = q.get()
    if(ethnicity == 'Error'):
        continue
    age = q.get()
    if(age == 'Error'):
        continue


    with open(output_folder + '/' + 'vgg.pickle', 'rb') as handle:
        vgg = pickle.load(handle)
        #print("++++++" + str(vgg))



    dataGotten = [['1','1',video_path,'age','ethnicity','gender',[],[],[],'language',[]]]
    df = pd.DataFrame(dataGotten,columns = ['ID','FACE_ID','VIDEO_PATH','caption_a','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','AUDIO_FEATURES','caption_l','image_path'])
    df["image_guide"] = np.nan
    df['low_res_image'] = df['image_path']

    df["caption_e"] = ethnicity
    df["caption_g"] = gender
    df["caption_a"] = age

    print("age :--------------"  + str(age))
    print("gender :--------------"  + str(gender))
    print("ethnicity :--------------"  + str(ethnicity))

    q.empty()

    #gpus
    import pickle



    # EXTRACT SPEAKER EMBEDDINGS, AUDIO FEATURES AND LANGUAGE

    audio_embs =  configParser.get('COMMON', 'audio_embs') 
    from pydub import AudioSegment
    import math
    # Get original duration of video
    audio = AudioSegment.from_file(video_path)
    audio_length_og = math.floor(audio.duration_seconds)


    video_filename = os.path.basename(video_path)

    absPathAudio_w = os.path.abspath(audio_folder) + "/" + video_filename
    absPathAudio = os.path.splitext(absPathAudio_w)[0]+'_audio.wav'
    absPathAudio_w = os.path.splitext(absPathAudio_w)[0]


    import subprocess
    # Extract audio monochannel and with 16khz and put it in absPathAudio

    command = "ffmpeg -nostats -loglevel 0 -y -i '" + video_path + "' -acodec pcm_s16le -ab 160k -ac 1 -ar 16000 -vn '" + absPathAudio  + "'"
    subprocess.call(command, shell=True)

    from threading import Thread
    import time

    # Function to delete audio temp files
    def delFiles(filesToDelete):
        time.sleep(ttwbdf)  # wait a bit
        for file in filesToDelete:  
            try:
                os.remove(file)
            except OSError:
                pass


    # Will either truncate or loop the original video to reach audio_length (6,12 or 24)
    audio_length_list = [24]
    for audio_length in audio_length_list:
        path_var_len_audio =  absPathAudio_w + "audio" + str(audio_length) + "s.wav"    # path to the variable length audio
        path_var_len_audio_temp =  absPathAudio_w + "audio_temp" + str(audio_length) + "s.wav"  # path to a temp version of the variable length audio

        if(audio_length_og > audio_length):
            # Truncate    

            command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + absPathAudio + "\" \"" + path_var_len_audio + "\""
            subprocess.call(command, shell=True)


        else:
            # Loop then truncaate
            #print("lesa")
            twoDigitLenStr = f"{audio_length:02}"
            #print(twoDigitLenStr)
            command = "ffmpeg -nostats -loglevel 0 -y -stream_loop -1 -i '" + absPathAudio + "' -t \"00:00:"+twoDigitLenStr+".000\" -codec:a \"aac\" -f \"wav\" -c copy '"+ path_var_len_audio_temp + "'"
            subprocess.call(command, shell=True)
            command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + path_var_len_audio_temp + "\" \"" + path_var_len_audio + "\""
            subprocess.call(command, shell=True)





        proc = multiprocessing.Process(target=extract_speechbrain_embeddings, args=(q,path_var_len_audio,))
        proc.start()
        proc.join()


        
        
        df['SPEAKER_EMB'] = q.get()
        df['caption_l'] = q.get()
        print(df['caption_l'])


        proc = multiprocessing.Process(target=extract_pyannote_titanet_embeddings, args=(q,path_var_len_audio,audio_embs,audio_length,openl3_mode,output_folder,))
        proc.start()
        proc.join()

        with open(output_folder + '/' + 'audio_emb_pyannote_titanet.pickle', 'rb') as handle:
            df['AUDIO_EMB'] = pickle.load(handle)

        proc = multiprocessing.Process(target=extract_audio_features, args=(q,path_var_len_audio,output_folder,))
        proc.start()
        proc.join()



        with open(output_folder + '/' + 'audio_features.pickle', 'rb') as handle:
            df['AUDIO_FEATURES'] = pickle.load(handle)

        ftd = [absPathAudio,path_var_len_audio,os.path.basename(path_var_len_audio),path_var_len_audio_temp]
        tDelete = Thread(target=delFiles, args=(ftd,))   # spawn a process
        tDelete.start()

    #proc = multiprocessing.Process(target=extract_vision_transformer, args=(q,output_folder,
    #                                                                        image_guide_path,1,
    #                                                                        image_size,
    #    boxBlurMin,boxBlurMax,gaussianBlurMin,gaussianBlurMax,pix_to_min,pix_to_max,))
    #proc.start()
    #proc.join()



    #with open(output_folder + '/' + 'image_features.pickle', 'rb') as handle:
    #    df["image_guide"] = pickle.load(handle)

    image = Image.open(image_guide_path).convert('RGB')

    w_s = image_size / (1+2 * 0.4)
    h_s = image_size / (1+2 * 0.4)

    image = image.crop((0.2*w_s, 0.0*h_s, 1.6*w_s, 1.4*h_s))

    image = image.resize((image_size,image_size))

    image = image.resize((begin_with_image_size,begin_with_image_size))
    im = image
    

    #print('saving')
    image.save(output_folder + "/" + "guide.png")

    #print(np.array(image,np.float32).shape)

    #pix = np.array(image, np.float32)
    #pix = np.moveaxis(pix, -1, 0)

    #pix = pix / 255
    image.close()
    im.close()
    #return pix.tolist()

    absPathGuide = absPathVideo.replace(datasetPathVideo,datasetPathGuides)
    absPathGuide = os.path.splitext(absPathGuide)[0]
    absPathGuide = absPathGuide + "_guide_"+ ".png"
    #print(absPathFace)
    #Create Directory
    pathlib.Path(os.path.dirname(absPathGuide)).mkdir(parents=True, exist_ok=True) 

    shutil.copy2(output_folder + "/" + "guide.png", absPathGuide) # complete target filename given


    df3 = df[["image_path","caption_a","caption_e","caption_g","caption_l"]]
    data_frame = df3
    data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
    data_frame['AUDIO_EMB'] = df['AUDIO_EMB']
    data_frame['AUDIO_FEATURES'] = df['AUDIO_FEATURES']
    data_frame['image_guide'] = df['image_guide']
    data_frame['low_res_image'] = df['low_res_image']

    import pickle


    with open(output_folder + '/' + 'filename.pickle', 'wb') as handle:
        pickle.dump(data_frame, handle)


    absPathFeatures = absPathVideo.replace(datasetPathVideo,datasetPathFeatures)
    absPathFeatures = os.path.splitext(absPathFeatures)[0]
    absPathFeatures = absPathFeatures + "_features_"+ ".pickle"
    #print(absPathFace)
    #Create Directory
    pathlib.Path(os.path.dirname(absPathFeatures)).mkdir(parents=True, exist_ok=True) 

    shutil.copy2(output_folder + "/" + "filename.pickle", absPathFeatures) # complete target filename given

    #data_frame.loc[index,"features_path"] = output_folder + '/' + 'filename.pickle'

    p_small = pathlib.Path(os.path.dirname(absPathVideo))
    p_big = p_small.parent.absolute()
    
    df_table = pd.DataFrame()
    df_table = df_table.append({'id': int(row['id']), 'video_path': row['video_path'], 
                    'face_path': absPathFace,'user':p_big.name,
        'blurred_face_path': absPathGuide, 'features_path': absPathFeatures, 
        'age': age,'gender':gender,'ethnicity':ethnicity,
        'vector' : vgg,'stage': 1}, ignore_index=True)
    

    if(i == 0):
        db.create_table("video_stage1", df_table)
    else:
        tbl = db.open_table("video_stage1")
        tbl.add(df_table)
        

    i = i + 1
    if(i == 20):
        break

    print('--------------------------------' + str(i) + '--------------------------------')

shutil.rmtree(output_folder)


Action: race: 100%|██████████| 3/3 [00:01<00:00,  2.15it/s]  


age :--------------36
gender :--------------Man
ethnicity :--------------white


/home/gamal/anaconda3/envs/ds2f_pre/lib/python3.8/site-packages/torch/functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:862.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


0    English
Name: caption_l, dtype: object


[NeMo W 2023-10-06 22:56:34 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-10-06 22:56:34 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


...

Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.1+rocm5. Bad things might happen unless you revert torch to 1.x.
Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.1+rocm5. Bad things might happen unless you revert torch to 1.x.
d


[NeMo W 2023-10-06 23:04:42 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-10-06 23:04:42 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-10-06 23:04:42 features:287] PADDING: 16
[NeMo I 2023-10-06 23:04:42 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/gamal/.cache/huggingface/hub/models--nvidia--speakerverification_en_titanet_large/snapshots/4e0b2d387a805da7c208b13d5898ee09de8ec1e9/speakerverification_en_titanet_large.nemo.
[NeMo I 2023-10-06 23:04:42 cloud:58] Found existing object /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
[NeMo I 2023-10-06 23:04:42 cloud:64] Re-using file from: /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo
[NeMo I 2023-10-06 23:04:42 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-10-06 23:04:42 features:287] PADDING: 16


[NeMo W 2023-10-06 23:04:42 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /ws/manifests/raid/combined/train_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
    augmentor:
      noise:
        manifest_path: /ws/manifests/raid/musan/musan_music_noise_manifest_dur8.json
        prob: 0.2
        min_snr_db: 5
        max_snr_db: 15
    num_workers: 4
    
[NeMo W 2023-10-06 23:04:42 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /ws/manifests/raid/voxceleb/small_manifest.json
    sample_rate: 16000
    labels: null
    batch_size:

[NeMo I 2023-10-06 23:04:42 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
e
f
g
h
i
j
--------------------------------11--------------------------------


In [16]:
tbl = db.open_table("video_stage1")
tbl.search(np.random.rand(2622)).limit(2).to_df()

,id,video_path,face_path,user,blurred_face_path,features_path,age,gender,ethnicity,vector,stage,_distance
0,6,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00009,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,30,Man,white,"[0.0135836145, 0.0042493055, 0.005943807, 0.01...",1,868.875854
1,4,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00010,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,26,Man,white,"[0.021446062, 0.010711044, 0.02926736, 0.02391...",1,868.960632


In [17]:
tbl.to_pandas()

,id,video_path,face_path,user,blurred_face_path,features_path,age,gender,ethnicity,vector,stage
0,0,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00002,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,36,Man,white,"[0.009602746, -0.004742456, 0.004785673, 0.032...",1
1,1,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00001,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,43,Man,white,"[0.0071346397, 0.0012256216, 0.008316233, 0.01...",1
2,2,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00006,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,27,Woman,white,"[0.009253787, 0.01768741, 0.006679517, 0.03707...",1
3,3,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00008,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,32,Man,middle eastern,"[0.015824728, 0.015256352, 0.0022940119, 0.022...",1
4,4,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00010,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,26,Man,white,"[0.021446062, 0.010711044, 0.02926736, 0.02391...",1
5,5,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00012,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,30,Woman,asian,"[0.011320298, 0.000799592, 0.0058022174, 0.019...",1
6,6,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00009,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,30,Man,white,"[0.0135836145, 0.0042493055, 0.005943807, 0.01...",1
7,7,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00005,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,33,Man,white,"[0.017614838, 0.009371514, 0.0011152884, 0.028...",1
8,8,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00004,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,25,Man,latino hispanic,"[0.019905446, 0.009860523, 0.006439571, 0.0109...",1
9,10,/home/gamal/Datasets/Dataset1Test/VideoTest/id...,/home/gamal/Datasets/Dataset1Test/FacesTest/id...,id00007,/home/gamal/Datasets/Dataset1Test/GuidesTest/i...,/home/gamal/Datasets/Dataset1Test/FeaturesTest...,26,Woman,asian,"[0.0077050775, 0.0055537038, 0.011315688, 0.01...",1
